# Solucion

Instalamos PySpark

In [ ]:
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Bajamos los datos:

In [ ]:
!wget https://github.com/datasciencedojo/datasets/raw/master/titanic.csv

--2023-08-04 23:43:29--  https://github.com/datasciencedojo/datasets/raw/master/titanic.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv [following]
--2023-08-04 23:43:29--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.001s  

2023-08-04 23:43:29 (39.1 MB/s) - ‘titanic.csv’ saved [60302/60302]



In [ ]:
# Creamos la sesion de Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

sc = spark.sparkContext

In [ ]:
import pandas as pd

df = pd.read_csv('titanic.csv')[['Pclass', 'Survived']]
raw_df = spark.createDataFrame(df)
raw_df.show()

+------+--------+
|Pclass|Survived|
+------+--------+
|     3|       0|
|     1|       1|
|     3|       1|
|     1|       1|
|     3|       0|
|     3|       0|
|     1|       0|
|     3|       0|
|     3|       1|
|     2|       1|
|     3|       1|
|     1|       1|
|     3|       0|
|     3|       0|
|     3|       0|
|     2|       1|
|     3|       0|
|     2|       1|
|     3|       0|
|     3|       1|
+------+--------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import VectorAssembler

# Vectorizamos todas las columnas numéricas en una sola columna de features.
# La API de MLlib requiere que usemos este formato.
feature_cols = raw_df.columns[:-1]  # Tomamos todas las columnas.
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
transformed_data = assembler.transform(raw_df)


In [ ]:
(train_data, test_data) = transformed_data.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features")
model = dt.fit(train_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="Survived",
    predictionCol="prediction",
    metricName="accuracy"
)

predictions = model.transform(test_data)

accuracy = evaluator.evaluate(predictions)
print(f"Accuracy = {round(accuracy * 100, 2)} %")

Accuracy = 65.64 %
